<a href="https://colab.research.google.com/github/leoagneau/Bib_Racer/blob/master/RBNR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Prepare drive

In [1]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')
# Set data path
#root_path = '/content/drive/My Drive/'
#proj_path = root_path + 'ML_Projects/Bib_Racer/'
!ln -s /content/drive/My\ Drive/ML_Projects/Bib_Racer /content/bib_racer
proj_path = '/content/bib_racer/'
data_path = proj_path + 'data/RBNR_data/datasets/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Load RNBR data

### Load & resize images

In [ ]:
import os
import cv2
from google.colab.patches import cv2_imshow  # Colab only, to replace cv2.imshow
img_dims={}
os.chdir(data_path)
orig_img_path = data_path + 'valid/'
yolo_img_path = data_path + 'yolo_valid/'
yolo_w = 416
yolo_h = 416
for file in os.scandir(orig_img_path):
  if (file.path.endswith(".JPG")):
    img = cv2.imread(file.path, cv2.IMREAD_UNCHANGED)
    img_dims[file.name.split(".")[0]] = [img.shape[1], img.shape[0]]  #[width, height]
    # Resize the image
    cv2.imwrite(yolo_img_path+file.name, cv2.resize(img, (yolo_w, yolo_h), interpolation=cv2.INTER_AREA))

### Load and process mat file, convert to Yolo txt file

In [ ]:
import numpy as np
from scipy.io import loadmat
lb_path = data_path + 'valid_labels/'
yolo_lb_path = data_path + 'yolo_train_labels/'
for file in os.scandir(lb_path):
  if (file.path.endswith(".mat")):
    m=loadmat(file)
    key = file.name.split(".")[0]
    print(key+':', img_dims[key])
    with open(yolo_lb_path+'/'+key+'.txt', 'w') as yolo_f:
      for f in m['facep']:
        f=f.astype(np.float, copy=False)
        xcenter = (f[3]+f[2])/(2*img_dims[key][0])  # (x1+x2)/(2*width)
        ycenter = (f[0]+f[1])/(2*img_dims[key][1])  # (y1+y2)/(2*height)
        width = (f[3]-f[2])/img_dims[key][0]  # (x2-x1)/(2*width)
        height = (f[1]-f[0])/img_dims[key][1]  # (y2-y1)/(2*height)
        print(0, xcenter, ycenter, width, height, file=yolo_f)
      for t in m['tagp']:
        t=t.astype(np.float, copy=False)
        xcenter = (t[3]+t[2])/(2*img_dims[key][0])  # (x1+x2)/(2*width)
        ycenter = (t[0]+t[1])/(2*img_dims[key][1])  # (y1+y2)/(2*height)
        width = (t[3]-t[2])/img_dims[key][0]  # (x2-x1)/(2*width)
        height = (t[1]-t[0])/img_dims[key][1]  # (y2-y1)/(2*height)
        print(1, xcenter, ycenter, width, height, file=yolo_f)

## Install Yolo & prepare to train custom objects
https://github.com/AlexeyAB/darknet/#how-to-train-to-detect-your-custom-objects

### Install Yolo AlexeyAB fork and prepare metadata

In [ ]:
%cd /content/
!rm -fr darknet
!git clone https://github.com/AlexeyAB/darknet/
%cd darknet

In [ ]:
%cd /content/darknet
#!mkdir build-release
#%cd build-release
#!cmake ..
#!make
#!make install
%cp "$proj_path"Makefile ./
!make

In [ ]:
!cp "$data_path"/yolo_cfg/yolo-obj.cfg /content/darknet/build/darknet/x64/cfg/
!cp "$data_path"/yolo_cfg/obj.names /content/darknet/build/darknet/x64/data/
!cp "$data_path"/yolo_cfg/obj.data /content/darknet/build/darknet/x64/data/
!cp "$data_path"/yolo_cfg/train.txt /content/darknet/build/darknet/x64/data/
!cp "$data_path"/yolo_cfg/valid.txt /content/darknet/build/darknet/x64/data/

In [ ]:
!cp /content/bib_racer/data/yolov3.weights /content/darknet/

In [ ]:
#%mkdir /content/darknet/build/darknet/x64/data/obj_train/
%mkdir /content/darknet/build/darknet/x64/data/obj_valid/
#!cp "$data_path"/yolo_train/*.* /content/darknet/build/darknet/x64/data/obj_train/ -v

In [ ]:
!cp "$proj_path"data/images/train_tiny_50files/*.* /content/darknet/build/darknet/x64/data/obj_valid/ -v

In [ ]:
#!wget -O "$proj_path"darknet53.conv.74 https://pjreddie.com/media/files/darknet53.conv.74

In [ ]:
#!cp "$data_path"/yolo_cfg/obj.data /content/darknet/build/darknet/x64/data/

###Train the model

In [ ]:
%cd /content/darknet/
#!./darknet detector train ./build/darknet/x64/data/obj.data ./build/darknet/x64/cfg/yolo-obj.cfg "$proj_path"darknet53.conv.74
!./darknet detector train ./build/darknet/x64/data/obj.data ./build/darknet/x64/cfg/yolo-obj.cfg /content/bib_racer/data/RBNR_data/weights_backup_resized_RBNR_training_images/yolo-obj_last.weights

## Testing YOLO with RBNR dataset

In [ ]:
# Prepare validation images and weight files
%cd /content/darknet/
![ ! -d /content/darknet/build/darknet/x64/data/obj_valid/ ] && mkdir /content/darknet/build/darknet/x64/data/obj_valid/ && cp "$data_path"/yolo_valid/*.* /content/darknet/build/darknet/x64/data/obj_valid/
WEIGHT_FILE = "yolo-obj_4000.weights"
![ ! -f $WEIGHT_FILE ] && cp "$proj_path"data/RBNR_data/weights_backup_resized_RBNR_training_images/$WEIGHT_FILE . -v

/content/darknet


In [ ]:
# Test
#!./darknet detector test ./build/darknet/x64/data/obj.data ./build/darknet/x64/cfg/yolo-obj.cfg $WEIGHT_FILE -thresh 0.25 -i 0 -ext_output
#!./darknet detector map ./build/darknet/x64/data/obj.data ./build/darknet/x64/cfg/yolo-obj.cfg $WEIGHT_FILE
!./darknet detector test ./cfg/coco.data ./cfg/yolov3.cfg ./yolov3.weights -thresh 0.25

## Face and Bib detection with Yolo
http://emaraic.com/blog/yolov3-custom-object-detector